# Automated CDR annotation using Anarci #
Installation procedure modified from the original repository https://github.com/oxpig/ANARCI. This notebooks utilizes a csv formatted dataset in the form (first chain, optional second chain, antigen, _, _, _, _, _, _) and replaces the _ entries by the CDR results


In [1]:
# install biopython
!pip install biopython==1.81
!conda install -c conda-forge biopython -y

# verifiying the installation's success
import Bio
print(Bio.__version__)
import sys
sys.path.append('/usr/local/lib/python3.12/site-packages')
from Bio import Align
print("BioPython Align module is working!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 21.6 MB/s eta 0:00:00
/bin/bash: line 1: conda: command not found
1.81
BioPython Align module is working!


In [2]:
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda init bash
!conda install -c conda-forge biopython -y
!conda install -c bioconda hmmer=3.3.2 -y
!git clone https://github.com/oxpig/ANARCI.git
%cd ANARCI
!python setup.py install
!find /usr/local -name "anarci" 2>/dev/null
!export PATH=$PATH:/usr/local/lib/python3.12/site-packages/anarci

--2025-03-04 13:16:39--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.32.241, 104.16.191.158, 2606:4700::6810:20f1, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.32.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 154615621 (147M) [application/octet-stream]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>] 147.45M   106MB/s    in 1.4s    

2025-03-04 13:16:40 (106 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [154615621/154615621]

PREFIX=/usr/local
Unpacking payload ...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPA

In [3]:
import pandas as pd
import sys
sys.path.append('/usr/local/lib/python3.12/site-packages')

from anarci import run_anarci

# Sample antibody sample to test CDR annotation by ground-truth comparison
sequences = [
    ("Heavy_Chain", "VKLLEQSGAEVKKPGASVKVSCKASGYSFTSYGLHWVRQAPGQRLEWMGWISAGTGNTKYSQKFRGRVTFTRDTSATTAYMGLSSLRPEDTAVYYCARDPYGGGKSEFDYWGQGTLVTVSS"),
    ("Light_Chain", "ELVMTQSPSSLSASVGDRVNIACRASQGISSALAWYQQKPGKAPRLLIYDASNLESGVPSRFSGSGSGTDFTLTISSLQPEDFAIYYCQQFNSYPLTFGGGTKVEIKRTV")
]
# Nanobody CDR annotation verified as well via the SABDAB database:
'''
sequences = [("Heavy_Chain", "QVQLVESGGALVQPGGSLRLSCAASGFPVNRYSMRWYRQADTNNDGWIEGDELKEREWVAGMSSAGDRSSYEDSVKGRFTISRDDARNTVYLQMNSLKPEDTAVYYCNVNVGFEYWGQGTQVTVSS"),]
'''


# Run ANARCI with Chothia numbering
results = run_anarci(sequences, scheme="chothia")

# Chothia CDR index definitions
# According to the official scheme
cdr_ranges = {
    "H1": (26, 32),
    "H2": (52, 56),
    "H3": (95, 102),
    "K1": (24, 34),
    "K2": (50, 56),
    "K3": (89, 97),
    "L1": (24, 34),
    "L2": (50, 56),
    "L3": (89, 97),
}

def extract_cdrs(results, cdr_ranges):
    """Extracts CDR sequences from ANARCI output based on Chothia numbering."""
    cdrs = {}
    for chain_idx, chain_data in enumerate(results[1]):  # Iterate over chains
        if not chain_data:  # Skip if empty
            continue

        chain_type = results[2][chain_idx][0]["chain_type"]  # Get 'H' or 'L'
        numbering = chain_data[0][0]  # Get the residue numbering info

        for cdr, (start, end) in cdr_ranges.items():
            print("cdr", cdr, cdr[0], chain_type)
            if cdr[0] == chain_type:  # Match H1, H2, H3 for heavy & L1, L2, L3 for light
                cdr_seq = ""
                print("numbering", numbering[0])
                for (num, ins), aa in numbering:
                    if start <= num <= end:  # Check if within CDR range
                        cdr_seq += aa  # Append amino acid (handling insertions)
                cdrs[cdr] = cdr_seq

    return cdrs

print("results", len(results), results)
# Extract CDRs
cdrs = extract_cdrs(results, cdr_ranges)

# Print the CDR sequences
for cdr, seq in cdrs.items():
    print(f"{cdr}: {seq}")

results 4 ([('Heavy_Chain', 'VKLLEQSGAEVKKPGASVKVSCKASGYSFTSYGLHWVRQAPGQRLEWMGWISAGTGNTKYSQKFRGRVTFTRDTSATTAYMGLSSLRPEDTAVYYCARDPYGGGKSEFDYWGQGTLVTVSS'), ('Light_Chain', 'ELVMTQSPSSLSASVGDRVNIACRASQGISSALAWYQQKPGKAPRLLIYDASNLESGVPSRFSGSGSGTDFTLTISSLQPEDFAIYYCQQFNSYPLTFGGGTKVEIKRTV')], [[([((1, ' '), 'V'), ((2, ' '), 'K'), ((3, ' '), 'L'), ((4, ' '), 'L'), ((5, ' '), 'E'), ((6, ' '), 'Q'), ((7, ' '), 'S'), ((8, ' '), 'G'), ((9, ' '), 'A'), ((10, ' '), 'E'), ((11, ' '), 'V'), ((12, ' '), 'K'), ((13, ' '), 'K'), ((14, ' '), 'P'), ((15, ' '), 'G'), ((16, ' '), 'A'), ((17, ' '), 'S'), ((18, ' '), 'V'), ((19, ' '), 'K'), ((20, ' '), 'V'), ((21, ' '), 'S'), ((22, ' '), 'C'), ((23, ' '), 'K'), ((24, ' '), 'A'), ((25, ' '), 'S'), ((26, ' '), 'G'), ((27, ' '), 'Y'), ((28, ' '), 'S'), ((29, ' '), 'F'), ((30, ' '), 'T'), ((31, ' '), 'S'), ((32, ' '), 'Y'), ((33, ' '), 'G'), ((34, ' '), 'L'), ((35, ' '), 'H'), ((36, ' '), 'W'), ((37, ' '), 'V'), ((38, ' '), 'R'), ((39, ' '), 'Q'), ((40, ' '), 'A'),

In [4]:
from google.colab import files
import os, shutil

# Prompt the user to upload a file
uploaded = files.upload()
!mkdir /content/datasets

# Get the filename of the uploaded file
for filename in uploaded.keys():
    source_dataset_filename = filename
    print(f"Uploaded file: {filename}")
    dest_path = os.path.join("/content/datasets", filename)
    shutil.move(filename, dest_path)
    output_filename = source_dataset_filename.replace(".csv", "_output.csv")
    print("source_filename", dest_path)
    print("output_filename", "/content/datasets" + output_filename)
    break

Saving abnb_generator_test_set_CDR_v5.csv to abnb_generator_test_set_CDR_v5.csv
Uploaded file: abnb_generator_test_set_CDR_v5.csv
source_filename /content/datasets/abnb_generator_test_set_CDR_v5.csv
output_filename /content/datasetsabnb_generator_test_set_CDR_v5_output.csv


In [5]:
# Load necessary libraries
import pandas as pd
import io, re
from contextlib import redirect_stdout, redirect_stderr
from tqdm import tqdm  # Progress bar

# Load CSV file into a DataFrame
df = pd.read_csv("/content/datasets/" + source_dataset_filename)

output_dataset = []

# Loop through all rows with tqdm progress bar
for index in tqdm(range(len(df)), desc="Processing Sequences", unit="seq"):
    if index + 10 >= len(df):  # Avoid index out of bounds
        break

    element = df.iloc[index + 10].tolist()  # Convert row to a list

    # Prepare sequences
    sequences = [("Heavy_Chain", re.sub(r'[^ACDEFGHIKLMNPQRSTVWY]', '', str(element[1])))]
    if len(str(element[2])) > 50:
        sequences.append(("Light_Chain", re.sub(r'[^ACDEFGHIKLMNPQRSTVWY]', '', str(element[2]))))
    else:
        if len(str(element[1])) < 50:
          continue
    try:
        # Run ANARCI while silencing output
        with io.StringIO() as f, io.StringIO() as e:
            with redirect_stdout(f), redirect_stderr(e):
                results = run_anarci(sequences, scheme="chothia")

        # Chothia CDR index definitions
        cdr_ranges = {
            "H1": (26, 32), "H2": (52, 56), "H3": (95, 102),
            "K1": (24, 34), "K2": (50, 56), "K3": (89, 97),
            "L1": (24, 34), "L2": (50, 56), "L3": (89, 97),
        }

        # Extract CDRs
        cdrs = extract_cdrs(results, cdr_ranges)

        # Process CDR sequences
        cdr_list = element[4:]
        for cdr_index, e in enumerate(cdr_list):
            if str(e).lower() == "nan":
                cdr_list[cdr_index] = ""

        if False:  # just for the initial usecase where some CDRs had to be truncated
            # Update CDR list with extracted sequences
            for cdr, seq in cdrs.items():
                for cdr_index in range(len(cdr_list)):
                    if seq.replace("-", "") in cdr_list[cdr_index]:
                        cdr_list[cdr_index] = seq.replace("-", "")
                        break
        else:  # If previously the dataset did not contain CDR annotations
            # Update CDR list with extracted sequences
            for cdr_index, (cdr, seq) in enumerate(cdrs.items()):
                # for cdr_index in range(len(cdr_list)):
                #     if seq in cdr_list[cdr_index]:
                cdr_list[cdr_index] = seq

        # Append processed data
        updated_element = [
            element[0], element[1], element[2], element[3],
            cdr_list[0], cdr_list[1], cdr_list[2], cdr_list[3],
            cdr_list[4], cdr_list[5]
        ]
        output_dataset.append(updated_element)
    except:
        print("could not determine CDRs for index " + str(index))

# save the dataset
df = pd.DataFrame(output_dataset)
df.to_csv("/content/datasets/" + output_filename, index=False, header=False)  # Save as CSV without index and headers

Processing Sequences:   0%|          | 0/543 [00:00<?, ?seq/s]

cdr H1 H H
numbering ((1, ' '), 'Q')
cdr H2 H H
numbering ((1, ' '), 'Q')
cdr H3 H H
numbering ((1, ' '), 'Q')
cdr K1 K H
cdr K2 K H
cdr K3 K H
cdr L1 L H
cdr L2 L H
cdr L3 L H
cdr H1 H K
cdr H2 H K
cdr H3 H K
cdr K1 K K
numbering ((1, ' '), 'D')
cdr K2 K K
numbering ((1, ' '), 'D')
cdr K3 K K
numbering ((1, ' '), 'D')
cdr L1 L K
cdr L2 L K
cdr L3 L K


Processing Sequences:   1%|          | 3/543 [00:00<00:25, 20.98seq/s]

cdr H1 H H
numbering ((1, ' '), 'Q')
cdr H2 H H
numbering ((1, ' '), 'Q')
cdr H3 H H
numbering ((1, ' '), 'Q')
cdr K1 K H
cdr K2 K H
cdr K3 K H
cdr L1 L H
cdr L2 L H
cdr L3 L H
cdr H1 H K
cdr H2 H K
cdr H3 H K
cdr K1 K K
numbering ((1, ' '), 'D')
cdr K2 K K
numbering ((1, ' '), 'D')
cdr K3 K K
numbering ((1, ' '), 'D')
cdr L1 L K
cdr L2 L K
cdr L3 L K
cdr H1 H H
numbering ((1, ' '), 'Q')
cdr H2 H H
numbering ((1, ' '), 'Q')
cdr H3 H H
numbering ((1, ' '), 'Q')
cdr K1 K H
cdr K2 K H
cdr K3 K H
cdr L1 L H
cdr L2 L H
cdr L3 L H
cdr H1 H L
cdr H2 H L
cdr H3 H L
cdr K1 K L
cdr K2 K L
cdr K3 K L
cdr L1 L L
numbering ((1, ' '), '-')
cdr L2 L L
numbering ((1, ' '), '-')
cdr L3 L L
numbering ((1, ' '), '-')
cdr H1 H H
numbering ((1, ' '), 'Q')
cdr H2 H H
numbering ((1, ' '), 'Q')
cdr H3 H H
numbering ((1, ' '), 'Q')
cdr K1 K H
cdr K2 K H
cdr K3 K H
cdr L1 L H
cdr L2 L H
cdr L3 L H
cdr H1 H H
numbering ((1, ' '), 'Q')
cdr H2 H H
numbering ((1, ' '), 'Q')
cdr H3 H H
numbering ((1, ' '), 'Q')
cdr 

Processing Sequences:   1%|          | 6/543 [00:00<00:22, 24.29seq/s]

...

cdr H1 H K
cdr H2 H K
cdr H3 H K
cdr K1 K K
numbering ((1, ' '), 'E')
cdr K2 K K
numbering ((1, ' '), 'E')
cdr K3 K K
numbering ((1, ' '), 'E')
cdr L1 L K
cdr L2 L K
cdr L3 L K
cdr H1 H H
numbering ((1, ' '), 'Q')
cdr H2 H H
numbering ((1, ' '), 'Q')
cdr H3 H H
numbering ((1, ' '), 'Q')
cdr K1 K H
cdr K2 K H
cdr K3 K H
cdr L1 L H
cdr L2 L H
cdr L3 L H
cdr H1 H K
cdr H2 H K
cdr H3 H K
cdr K1 K K
numbering ((1, ' '), 'D')
cdr K2 K K
numbering ((1, ' '), 'D')
cdr K3 K K
numbering ((1, ' '), 'D')
cdr L1 L K
cdr L2 L K
cdr L3 L K
cdr H1 H H
numbering ((1, ' '), 'E')
cdr H2 H H
numbering ((1, ' '), 'E')
cdr H3 H H
numbering ((1, ' '), 'E')
cdr K1 K H
cdr K2 K H
cdr K3 K H
cdr L1 L H
cdr L2 L H
cdr L3 L H
cdr H1 H K
cdr H2 H K
cdr H3 H K
cdr K1 K K
numbering ((1, ' '), 'D')
cdr K2 K K
numbering ((1, ' '), 'D')
cdr K3 K K
numbering ((1, ' '), 'D')
cdr L1 L K
cdr L2 L K
cdr L3 L K
cdr H1 H H
numbering ((1, ' '), 'Q')
cdr H2 H H
numbering ((1, ' '), 'Q')
cdr H3 H H
numbering ((1, ' '), 'Q')
cdr 

Processing Sequences:  97%|█████████▋| 529/543 [00:25<00:00, 23.89seq/s]

cdr H1 H H
numbering ((1, ' '), '-')
cdr H2 H H
numbering ((1, ' '), '-')
cdr H3 H H
numbering ((1, ' '), '-')
cdr K1 K H
cdr K2 K H
cdr K3 K H
cdr L1 L H
cdr L2 L H
cdr L3 L H
cdr H1 H K
cdr H2 H K
cdr H3 H K
cdr K1 K K
numbering ((1, ' '), 'Y')
cdr K2 K K
numbering ((1, ' '), 'Y')
cdr K3 K K
numbering ((1, ' '), 'Y')
cdr L1 L K
cdr L2 L K
cdr L3 L K
cdr H1 H H
numbering ((1, ' '), '-')
cdr H2 H H
numbering ((1, ' '), '-')
cdr H3 H H
numbering ((1, ' '), '-')
cdr K1 K H
cdr K2 K H
cdr K3 K H
cdr L1 L H
cdr L2 L H
cdr L3 L H
cdr H1 H K
cdr H2 H K
cdr H3 H K
cdr K1 K K
numbering ((1, ' '), 'Y')
cdr K2 K K
numbering ((1, ' '), 'Y')
cdr K3 K K
numbering ((1, ' '), 'Y')
cdr L1 L K
cdr L2 L K
cdr L3 L K
cdr H1 H H
numbering ((1, ' '), '-')
cdr H2 H H
numbering ((1, ' '), '-')
cdr H3 H H
numbering ((1, ' '), '-')
cdr K1 K H
cdr K2 K H
cdr K3 K H
cdr L1 L H
cdr L2 L H
cdr L3 L H
cdr H1 H K
cdr H2 H K
cdr H3 H K
cdr K1 K K
numbering ((1, ' '), 'D')
cdr K2 K K
numbering ((1, ' '), 'D')
cdr K3 K

Processing Sequences:  98%|█████████▊| 533/543 [00:25<00:00, 20.55seq/s]

cdr H1 H H
numbering ((1, ' '), 'E')
cdr H2 H H
numbering ((1, ' '), 'E')
cdr H3 H H
numbering ((1, ' '), 'E')
cdr K1 K H
cdr K2 K H
cdr K3 K H
cdr L1 L H
cdr L2 L H
cdr L3 L H
cdr H1 H L
cdr H2 H L
cdr H3 H L
cdr K1 K L
cdr K2 K L
cdr K3 K L
cdr L1 L L
numbering ((1, ' '), '-')
cdr L2 L L
numbering ((1, ' '), '-')
cdr L3 L L
numbering ((1, ' '), '-')
cdr H1 H K
cdr H2 H K
cdr H3 H K
cdr K1 K K
numbering ((1, ' '), 'D')
cdr K2 K K
numbering ((1, ' '), 'D')
cdr K3 K K
numbering ((1, ' '), 'D')
cdr L1 L K
cdr L2 L K
cdr L3 L K
cdr H1 H H
numbering ((1, ' '), 'E')
cdr H2 H H
numbering ((1, ' '), 'E')
cdr H3 H H
numbering ((1, ' '), 'E')
cdr K1 K H
cdr K2 K H
cdr K3 K H
cdr L1 L H
cdr L2 L H
cdr L3 L H
cdr H1 H H
numbering ((1, ' '), 'E')
cdr H2 H H
numbering ((1, ' '), 'E')
cdr H3 H H
numbering ((1, ' '), 'E')
cdr K1 K H
cdr K2 K H
cdr K3 K H
cdr L1 L H
cdr L2 L H
cdr L3 L H
cdr H1 H K
cdr H2 H K
cdr H3 H K
cdr K1 K K
numbering ((1, ' '), 'D')
cdr K2 K K
numbering ((1, ' '), 'D')
cdr K3 K